In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the gesture recognizer model
model = load_model('C:/Users/omgsy/Downloads/hand-gesture-recognition-code/mp_hand_gesture')
# Load class names
f = open('C:/Users/omgsy/Downloads/hand-gesture-recognition-code/gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']


In [6]:
import cv2
import mediapipe as mp
import time
import numpy as np
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils
circleDrawingSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=1, color=(0,255,0))
lineDrawingSpec = mpDraw.DrawingSpec(thickness=1, color=(0,255,0))

pTime = 0
cTime = 0



while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    className = ''
    landmarks=[]
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                #print(id,lm)
                w, h, c = img.shape
                cx= int(lm.x *w)
                cy=int(lm.y*h)
                landmarks.append([cx,cy])
                print(landmarks)

                cv2.circle(img, (cx,cy), 3, (0,255,0), cv2.FILLED)

                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS,circleDrawingSpec,lineDrawingSpec)
                
                prediction = model.predict([landmarks])

                classID = np.argmax(prediction)
                className = classNames[classID]
  # show the prediction on the frame
    cv2.putText(img, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img,str(int(fps)), (150,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)

    cv2.imshow("Image", img)
    
    cv2.waitKey(1)
    

[[453, 329]]


ValueError: in user code:

    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\omgsy\anaconda3\envs\roboticLab3\lib\site-packages\keras\engine\